<a href="https://colab.research.google.com/github/AlyShmahell/WeirdTargets/blob/master/weirdtargets2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
! pip install ujson

In [1]:
import os
import gc
import sys
import ujson
import psutil
import requests
import numpy     as     np
import pandas    as     pd
import operator  as     op
from   copy      import copy
from   tqdm      import tqdm
from   functools import reduce
from   gzip      import GzipFile
from   toolz     import partition_all
from   itertools import islice
from   pathlib   import Path
from   collections import Counter
from   sklearn.cluster import KMeans
from   concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

# nCr

In [3]:
def ncr(n, r):
    r     = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return  numer / denom

# Downloader

In [4]:
def downloader(url):
    def func(r, path, filename, total_size, block_size):
        t          = tqdm(
            desc       = f"downloading {filename}",
            total      = total_size, 
            unit       = 'iB', 
            unit_scale = True
        )
        with open(path/filename, 'wb') as f:
            for data in r.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
        try:
            assert not(total_size != 0 and t.n != total_size)
        except:
            sys.exit(f"downloaded {t.n}/{total_size}", flush=True)
    r          = requests.get(url, stream=True)
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024**2
    filename   = url.split('/')[-1]
    path       = Path(Path.home())/Path('workspace')
    path.mkdir(parents=True, exist_ok=True)
    try:
        assert Path(path/filename).stat().st_size == total_size
        print(f"{filename} is already downloaded", flush=True)
    except:
        func(r, path, filename, total_size, block_size)
    filename    = filename.split('.')
    (filename, 
     extension) = ".".join(filename[:-1]), filename[-1]
    return path, filename, extension

In [5]:
class Prerocessor:
    def __traverse__(self, parsed, keys):
        if len(keys)>1:
            return self.__traverse__(parsed[keys[0]], keys[1:])
        return parsed[keys[0]]
    def __json__(self, string):
        parsed = ujson.loads(string)
        obj = {
            key: self.__traverse__(parsed, self.keygroup[key])
            for key in self.keygroup.keys()
        }
        return obj
    def __df__(self, batch):
        json = map(lambda string: self.__json__(string), batch)
        df   = pd.DataFrame.from_records(json, 
                                         columns=["target",
                                                  "disease",
                                                  "score"]
        )
        return df
    def __init__(self, url, keygroup):
        (path, 
         filename, 
         extension)   = downloader(url)
        self.keygroup = keygroup
        """slices = [0]
        size    = 0
        bs      = int(12.8e+7)
        with tqdm(desc=f"Feature Probing") as tqdmo:
            with GzipFile(f"{path}/{filename}.{extension}") as f:
                for i, b in enumerate(f):
                    if size > bs:
                        slices += [i-1]
                        size    = 0
                    size += (33+len(b))
                    tqdmo.update(1)
                slices += [i]
        slices = [
                  jdx - idx
                  for idx, jdx in zip(slices[:-1], slices[1:])
        ]
        n = max(slices)"""
        n = 952457
        with tqdm(desc=f"Feature Extraction, n={n}") as tqdmo:
            with GzipFile(f"{path}/{filename}.{extension}") as f:
                batches = partition_all(n, f)
                for batch in batches:
                    df    = self.__df__(batch)
                    df.to_hdf(
                        f"{path}/{filename}.hdf",
                        key='features',
                        mode='a',
                        format='table',
                        append=True
                    )
                    tqdmo.update(n)
        self.df = pd.read_hdf(
                f"{path}/{filename}.hdf",
                key='features'
            )
        print(len(self.df.index))


In [ ]:
%timeit
Prerocessor(
    'https://storage.googleapis.com/open-targets-data-releases/17.12/17.12_evidence_data.json.gz',
    {
        "target" : ["target", "id"],
        "disease": ["disease", "id"],
        "score":   ["scores", "association_score"]
    }
)

17.12_evidence_data.json.gz is already downloaded


Feature Extraction, n=952457: 952457it [01:55, 8273.33it/s]

In [ ]:
!ls -al /root/workspace

In [ ]:

!rm -rf /root/workspace/*.hdf
!ls /root/workspace

In [ ]:
S = []
with tqdm(desc=f"Feature Probing") as tqdmo:
     with GzipFile(f"/root/workspace/17.12_evidence_data.json.gz") as f:
        for b in f:
            S += [len(b)+33]
            tqdmo.update(1)
with open('lens.json', 'w') as f:
    ujson.dump(S, f)

In [6]:
with open('lens.json', 'r') as f:
    S = np.array(ujson.load(f))
    print(S)


def max_subarray(S):
    numbers = S.copy()
    mean = np.mean(numbers)
    numbers = numbers - mean
    """Find a contiguous subarray with the largest sum."""
    best_sum = 0  # or: float('-inf')
    best_start = best_end = 0  # or: None
    current_sum = 0
    for current_end, x in enumerate(numbers):
        if current_sum <= 0:
            # Start a new sequence at the current element
            current_start = current_end
            current_sum = x
        else:
            # Extend the existing sequence with the current element
            current_sum += x

        if current_sum > best_sum and current_sum+((current_end-current_start)*mean) < int(6040779371/2):
            best_sum = current_sum
            best_start = current_start
            best_end = current_end + 1  # the +1 is to make 'best_end' exclusive

    return best_sum, best_start, best_end

_, a, b = max_subarray(S)
print(sum(S[a:b]), b-a)

[2525 2440 2623 ... 6565 5437 6160]
3020393697 475395
